In [1]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import Calendar
import os
from datetime import datetime
import numpy as np
import re
import time
import folium
from folium.plugins import MarkerCluster
import webbrowser

In [2]:
def convertir_a_decimal(coordenada):
    # Usar expresiones regulares para extraer grados, minutos y segundos
    match = re.match(r"(\d+)°(\d+)'(\d+)\"", coordenada)
    
    if match:
        grados, minutos, segundos = map(float, match.groups())
        decimal = -1*(grados + minutos / 60 + segundos / 3600)
        return round(decimal, 7)
    else:
        # Manejar casos donde el formato no es el esperado
        print(f"Error en el formato de la coordenada: {coordenada}")
        return None  # O manejar de otra manera según tus necesidades

In [3]:
empresas=["MultiX", "Blumar", "Yadran", "Australis", "Aquachile", "SalmonesAustral","Camanchaca","Mowi", "Cermaq", "GMT"]
profundidad=[0, 5, 10]

In [4]:
### SELECCIÓN DE FECHA ###
fecha_seleccionada = None
def show_calendar():
    def on_date_select():
        global fecha_seleccionada
        fecha_seleccionada = cal.get_date()
        top.destroy()
        main_window.destroy()

    top = tk.Toplevel(main_window)
    cal = Calendar(top, font="Arial 14", selectmode='day', locale='es_ES')
    cal.pack(fill="both", expand=True)
    ttk.Button(top, text="Seleccionar", command=on_date_select).pack()

main_window = tk.Tk()
main_window.config(width=300, height=200)
main_window.title("Seleccionar Fecha")

button = ttk.Button(text="Seleccionar Fecha", command=show_calendar)
button.place(x=50, y=50)
main_window.mainloop()
fecha_obj = datetime.strptime(fecha_seleccionada, "%d/%m/%y")
fecha=fecha_obj.strftime("%Y-%m-%d %H:%M:%S")

In [7]:
### BASE DE DATOS ALGAS NOCIVAS ###
conc_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\Algas_Nocivas.xlsx"
df_algas_nocivas = pd.read_excel(conc_path, sheet_name="CENTRO")

### BASE DE DATOS DE UBICACIONES GEOGRÁFICAS DE CENTROS ###
centros_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\Centros.xlsx"
df_empresas={}
for i in empresas:
    df_centros=pd.read_excel(centros_path,sheet_name=i)
    df_empresas[f"df_centros_{i}"]=df_centros
df_centros_MultiX=df_empresas["df_centros_MultiX"]
df_centros_Blumar=df_empresas["df_centros_Blumar"]
df_centros_MultiX['LATITUDE'] = df_centros_MultiX['LATITUDE'].apply(convertir_a_decimal)
df_centros_MultiX['LONGITUDE'] = df_centros_MultiX['LONGITUDE'].apply(convertir_a_decimal)
df_centros_MultiX.head()

TypeError: expected string or bytes-like object

In [ ]:
DB_path=r"C:\Users\ernes\Documents\Trabajo\Proyecto_Beta\datos_xlsx\DB.xlsx"
df_DB=pd.read_excel(DB_path,sheet_name="MultiX")
df_DB_filtrada=df_DB[
    (df_DB['MONITOREO'] == "CENTRO") &
    (df_DB['NOCIVO'] != 'NO') &
    (df_DB['FECHA'] == fecha)
]
especies = df_DB_filtrada['ESPECIE'].unique().tolist() #Lista con especies registradas al la fecha

#especies = [especie.replace(' ', '_').replace('.', '_') for especie in especies]

In [ ]:
df_DB_filtrada[(df_DB_filtrada['ESPECIE']==especies[0])&(df_DB_filtrada['PROFUNDIDAD']==5)]['CANTIDAD']

5        80000
35       40000
45       60000
57       66000
84     1800000
163     800000
Name: CANTIDAD, dtype: int64

In [ ]:
capas={}
for especie in especies:
    for i in profundidad:
        capa_especie_profundidad = folium.FeatureGroup(name=f'{especie}_{i}m')
        capas[f"{especie}_{i}m_{fecha_seleccionada.replace('/','-')}"]=capa_especie_profundidad
        lista_centros=df_DB_filtrada[(df_DB_filtrada['ESPECIE']==especie)&(df_DB_filtrada['PROFUNDIDAD']==i)]['CENTRO'].tolist()
        cantidad=df_DB_filtrada[(df_DB_filtrada['ESPECIE']==especie)&(df_DB_filtrada['PROFUNDIDAD']==i)]['CANTIDAD']  
        if cantidad <= df_algas_nocivas[df_algas_nocivas['ESPECIE']==especie]['NORMAL']:
            folium.Circle(location=[-42.7128, -74.0060], color="lightgreen", fill_color="green", radius=1500, fill_opacity=0.5).add_to(capa_circulos)

print(capas)

ValueError: Can only compare identically-labeled Series objects

In [ ]:
df_algas_nocivas[df_algas_nocivas['ESPECIE']==especies[0]]['NORMAL']

11    50000.0
Name: NORMAL, dtype: float64

In [ ]:
# Crea un mapa centrado en una ubicación específica
attr = (
    'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
mapa = folium.Map(location=[-47.7128, -74.0060], zoom_start=5.8, tiles=tiles, attr=attr)

# Crear un grupo de capas para los círculos
capa_circulos = folium.FeatureGroup(name='Círculos')
# Crear un conjunto específico llamado "Alerta"
capa_alerta = folium.FeatureGroup(name='Alerta')


# Añadir círculos a la capa
folium.Circle(location=[-43.7128, -74.0060], color="red", fill_color="red", radius=500, fill_opacity=0.5).add_to(capa_circulos)
folium.Circle(location=[-42.7128, -74.0060], color="yellowgreen", fill_color="green", radius=500, fill_opacity=0.5).add_to(capa_circulos)
folium.Circle(location=[-44.7128, -74.0060], color="yellow", fill_color="yellow", radius=500, fill_opacity=0.5).add_to(capa_circulos)

# Añadir círculos rojo y amarillo al conjunto "Alerta"
folium.Circle(location=[-43.7128, -74.0060], color="red", fill_color="red", radius=500, fill_opacity=0.5).add_to(capa_alerta)
folium.Circle(location=[-44.7128, -74.0060], color="yellow", fill_color="yellow", radius=500, fill_opacity=0.5).add_to(capa_alerta)

# Añadir las capas al mapa
capa_circulos.add_to(mapa)
capa_alerta.add_to(mapa)

# Añadir control de capas al mapa
folium.LayerControl().add_to(mapa)
# Guarda el mapa como un archivo HTML
mapa_nombre='mapa.html'
mapa.save(mapa_nombre)
webbrowser.open(mapa_nombre)

True